<a href="https://colab.research.google.com/github/MrBasM/TensorFlow-for-Deep-Learning-Bootcamp/blob/main/00_tenserflow_fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# In this notebook, we're going to cover some of the most fundatmental concepts of tensors using TensorFlow

More specifically, we're going to cover:
* Introduction to tensors
* Getting information from tensors
* Manipulating tensors
* Tensors & NumPy
* Using @tf.function (a way to speed up your regular Python functions)
* Using GPUs with TensorFlow (or TPUs)
* Excercises to try for yourself!

## Introduction to Tensors

In [ ]:
# Import TensorFlow
import tensorflow as tf
import numpy as np
print(tf.__version__)

2.18.0


In [ ]:
# Create tensors with tf.constant()
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [ ]:
# Check the number of dimensions of a tensor (ndim stands for the number of dimensions)
scalar.ndim

0

In [ ]:
# Create a vector
vector = tf.constant([10, 10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [ ]:
# Check the dimension of our vector
vector.ndim

1

In [ ]:
# Create a matrix (has more than 1 dimension)
matrix = tf.constant([[10, 7],
                      [7, 10]])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]], dtype=int32)>

In [ ]:
matrix.ndim

2

In [ ]:
# Create another matrix
another_matrix = tf.constant([[10., 7.],
                              [3., 2.],
                              [8., 9.]], dtype=tf.float16) # specify the data type with dtype parameter
another_matrix

<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[10.,  7.],
       [ 3.,  2.],
       [ 8.,  9.]], dtype=float16)>

In [ ]:
# What's the number dimensions of another_matrix?
another_matrix.ndim

2

In [ ]:
# Let's create a tensor
tensor = tf.constant([[[1,2,3],
                       [4,5,6]],
                      [[7,8,9],
                       [10,11,12]],
                      [[13,14,15],
                       [16,17,18]]])
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]], dtype=int32)>

In [ ]:
tensor.ndim

3

What we've created so far:

* Scalar: a single number
* Vector: a number with direction (e.g. wind speed and direction)
* Matrix: a 2-dimensional array of numbers
* Tensor: a n-dimensional array of numbers (when n can be any number, a 0-dimensional tensor is a scalar, a 1-dimensional tensor is a vector)

### Creating tensors with `tf.Variable`


In [ ]:
# Create the same tensor with tf.Variable() as above
changeable_tensor = tf.Variable([10,7])
unchangable_tensor = tf.constant([10,7])
changeable_tensor, unchangable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>)

In [ ]:
# Let's try change one of the elements in our changeable tensor

# changeable_tensor[0] = 7
# changeable_tensor

# returns error TypeError: 'ResourceVariable' object does not support item assignment


In [ ]:
# How about we try .assign()
changeable_tensor[0].assign(7)
changeable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([7, 7], dtype=int32)>

In [ ]:
# Let's try change our unchangeable tensor

# unchangable_tensor[0].assign(7)

# AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'assign'

🔑 **Note:** Rarely in practice will you need to decide whether to use `tf.constant` or `tf.variable` to create tensors, as TensorFlow does this for you. However, if in doubt, use `tf.constant` and change it later if needed.

### Creating random tensors

Random tensors are tensors of some abitary size which contain random numbers.

In [ ]:
# Let's create two random (but the same) tensors
random_1 = tf.random.Generator.from_seed(42) # Set seed for reproducibility
random_1 = random_1.normal(shape=(3,2))
random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.normal(shape=(3,2))

# Are these equal
random_1, random_2, random_1 == random_2

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193765, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193765, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True]])>)

### Shuffle the order of elements in a tensor


In [ ]:
# Shuffle a tensor (valuable for when you want to shuffle your data so the inherent order doesn't effect learning)
not_shuffled = tf.constant([[10,7],
                            [3,4],
                            [2,5]])
# Shuffle our non-shuffled tensor
tf.random.shuffle(not_shuffled)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 3,  4],
       [10,  7],
       [ 2,  5]], dtype=int32)>

In [ ]:
# Shuffle our non-shuffled tensor
tf.random.set_seed(42)
tf.random.shuffle(not_shuffled, seed=42)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

🛠️ ***Exercise:*** Read through TensorFlow documentation on random see generation: https://www.tensorflow.org/api_docs/python/tf/random/set_seed and practice writing 5 random tensors and shuffle them.

It looks like if we want our shuffled tensors to be in the same order, we've got to use the global level random seed as well as the operation level random seed:

> Rule 4: if both the global and the operation seed are set: Both seeds are used in conjenction to determine the random sequence.

In [ ]:
# create 5 random tensors
tf.random.set_seed(42)
tensor_generator = tf.random.Generator.from_seed(42)

tensor_1 = tf.constant([[10,7],
                            [3,4],
                            [2,5]])
tensor_2 = tensor_generator.normal(shape=(3,2))
tensor_3 = tf.constant(np.random.rand(3,2))
tensor_4 = tf.random.uniform(shape=(3,2))
tensor_5 = tf.random.normal(shape=(10,7))

# shuffle them
tf.random.shuffle(tensor_1)
tf.random.shuffle(tensor_2)
tf.random.shuffle(tensor_3)
tf.random.shuffle(tensor_4)
tf.random.shuffle(tensor_5)

tensor_1, tensor_2, tensor_3, tensor_4, tensor_5

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[10,  7],
        [ 3,  4],
        [ 2,  5]], dtype=int32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193765, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float64, numpy=
 array([[0.40607483, 0.8429172 ],
        [0.36152697, 0.74527042],
        [0.04343779, 0.14814282]])>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[0.6645621 , 0.44100678],
        [0.3528825 , 0.46448255],
        [0.03366041, 0.68467236]], dtype=float32)>,
 <tf.Tensor: shape=(10, 7), dtype=float32, numpy=
 array([[ 8.4224582e-02, -8.6090374e-01,  3.7812304e-01, -5.1962738e-03,
         -4.9453196e-01,  6.1781919e-01, -3.3082047e-01],
        [-1.3840806e-03, -4.2373410e-01, -1.3872087e+00, -1.5488191e+00,
         -5.3198391e-01, -4.4756433e-01, -2.0115814e+00],
        [-5.7926011e-01,  5.7938921e-01,  1.3041967e+00,  6.7720258e-01,


In [ ]:
tf.random.set_seed(42) # Global level random seed
tf.random.shuffle(not_shuffled, seed=42) # operation level random seed

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

In [ ]:
tf.random.set_seed(42) # Global level random seed
tf.random.shuffle(not_shuffled, seed=42) # operation level random seed

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

In [ ]:
# Create a tensor of all ones
tf.ones([10,7])

<tf.Tensor: shape=(10, 7), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>

In [ ]:
# Create a tensor of all zeros
tf.zeros(shape=(10,7))

<tf.Tensor: shape=(10, 7), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

### Turn NumPy array into tensors

The main difference between NumPy arrays and TensorFLow tensors is that tensors can be run on a GPU (much faster for nummerical computing).

In [ ]:
# You can also turn NumPy arrays into tensors
import numpy as np
numpy_A = np.arange(1, 25, dtype=np.int32) # create a Numpy array between 1 and 25
numpy_A

# X = tf.constant(some_matrix) # capital for matrix or tensor
# y = tf.constant(vector) # non-capital for vector

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)

In [ ]:
A = tf.constant(numpy_A, shape = (2,3,4))
B = tf.constant(numpy_A)
A, B

(<tf.Tensor: shape=(2, 3, 4), dtype=int32, numpy=
 array([[[ 1,  2,  3,  4],
         [ 5,  6,  7,  8],
         [ 9, 10, 11, 12]],
 
        [[13, 14, 15, 16],
         [17, 18, 19, 20],
         [21, 22, 23, 24]]], dtype=int32)>,
 <tf.Tensor: shape=(24,), dtype=int32, numpy=
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24], dtype=int32)>)

### Getting information from tensors

When dealing with tensors you probably want to be aware of the following attributes:
* Shape
* Rank
* Axis or dimension
* Size

In [ ]:
# Create a rank 4 tensor (4 dimensions)
rank_4_tensor = tf.zeros(shape=[2,3,4,5])
rank_4_tensor

<tf.Tensor: shape=(2, 3, 4, 5), dtype=float32, numpy=
array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]],


       [[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]]], dtype=float32)>

In [ ]:
rank_4_tensor.shape, rank_4_tensor.ndim, tf.size(rank_4_tensor)

(TensorShape([2, 3, 4, 5]), 4, <tf.Tensor: shape=(), dtype=int32, numpy=120>)

In [ ]:
# calculate the size
2 * 3 * 4 * 5

120

In [ ]:
# Get various attributes of our tensor
print("Datatype of every element: ", rank_4_tensor.dtype)
print("Number of dimensions (rank): ", rank_4_tensor.ndim)
print("Shape of tensor: ", rank_4_tensor.shape)
print("Elements along the 0 axis: ", rank_4_tensor.shape[0])
print("ELements along the last axis: ", rank_4_tensor.shape[-1])
print("Total number of elements in our tensor: ", tf.size(rank_4_tensor))
print("Total number of elements in our tensor: ", tf.size(rank_4_tensor).numpy())

Datatype of every element:  <dtype: 'float32'>
Number of dimensions (rank):  4
Shape of tensor:  (2, 3, 4, 5)
Elements along the 0 axis:  2
ELements along the last axis:  5
Total number of elements in our tensor:  tf.Tensor(120, shape=(), dtype=int32)
Total number of elements in our tensor:  120


### Indexing tensors

Tensors can be indexed just like Python lists.

In [ ]:
some_list = [1,2,3,4]
some_list[:2]

[1, 2]

In [ ]:
# Get the first 2 elements of each dimension
rank_4_tensor[:2, :2, :2, :2]

<tf.Tensor: shape=(2, 2, 2, 2), dtype=float32, numpy=
array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]]], dtype=float32)>

In [ ]:
# Get the first element from each dimension from each index except for the final one
rank_4_tensor[:1, :1, :1, :]

<tf.Tensor: shape=(1, 1, 1, 5), dtype=float32, numpy=array([[[[0., 0., 0., 0., 0.]]]], dtype=float32)>

### Manipulating tensors (tensor operations)

**Basic operations**

`+`,`-`,`*`,`/`

In [ ]:
# You can add values to a tensor using the addition operator
tensor = tf.constant([[10,7],[3,4]])
tensor + 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]], dtype=int32)>

In [ ]:
tensor * 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]], dtype=int32)>

In [ ]:
tensor - 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 0, -3],
       [-7, -6]], dtype=int32)>

In [ ]:
tensor / 2

<tf.Tensor: shape=(2, 2), dtype=float64, numpy=
array([[5. , 3.5],
       [1.5, 2. ]])>

In [ ]:
# We can use the tensorflow builtin funtion too
tf.multiply(tensor, 10)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]], dtype=int32)>

In [ ]:
tf.subtract(tensor, 10)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 0, -3],
       [-7, -6]], dtype=int32)>

In [ ]:
tf.add(tensor, 10)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]], dtype=int32)>

In [ ]:
tf.divide(tensor, 10)

<tf.Tensor: shape=(2, 2), dtype=float64, numpy=
array([[1. , 0.7],
       [0.3, 0.4]])>

**Matrix multiplication**

In machine learning, matrix multiplication is one of the most common tensor operations.

There are two rules our tensors (or matrices) need to fulfill if we're going to matrix multiply them:
1. The innner dimensions must match
2. The resulting matrix has the shape of the outer dimensions

📖 **Resource:** Info and example of matrix multiplication: https://www.mathsisfun.com/algebra/matrix-multiplying.html

In [ ]:
# Matrix multiplication in TensorFlow
print(tensor)
tf.matmul(tensor, tensor)

tf.Tensor(
[[10  7]
 [ 3  4]], shape=(2, 2), dtype=int32)


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]], dtype=int32)>

In [ ]:
# Matrix multoplocation with Python operator "@"
tensor @ tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]], dtype=int32)>

In [ ]:
# excercise
tensor_a = tf.constant([[1,2,5],
              [7,2,1],
              [3,3,3]])
tensor_b = tf.constant([[3,5],
                         [6,7],
                         [1,8]])
print(tensor_a)
print(tensor_b)

tf.matmul(tensor_a, tensor_b)

tf.Tensor(
[[1 2 5]
 [7 2 1]
 [3 3 3]], shape=(3, 3), dtype=int32)
tf.Tensor(
[[3 5]
 [6 7]
 [1 8]], shape=(3, 2), dtype=int32)


<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[20, 59],
       [34, 57],
       [30, 60]], dtype=int32)>

In [ ]:
tensor_a @ tensor_b

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[20, 59],
       [34, 57],
       [30, 60]], dtype=int32)>

In [ ]:
# Create a (3,2) tensor
X = tf.constant([[1,2],
                 [3,4],
                 [5,6]])
# Create another (3,2) tensor
Y = tf.constant([[7,8],
                 [9,10],
                 [11,12]])
X, Y

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>,
 <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[ 7,  8],
        [ 9, 10],
        [11, 12]], dtype=int32)>)

In [ ]:
# Try to multiply tensors of same shape
# This does't work (3,2) (3,2), the inner dimensions don't match 2 3
# X @ Y

In [ ]:
# Let's change the shape of Y
tf.reshape(Y, shape=(2,3))

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[ 7,  8,  9],
       [10, 11, 12]], dtype=int32)>

In [ ]:
# Try to matrix multiply X by reshaped Y
X @ tf.reshape(Y, shape=(2,3))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [ ]:
tf.matmul(X,tf.reshape(Y, shape=(2,3)))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [ ]:
# Can do the same with transpose, but it's not exactly the same. The values are placed differently. Transpose flips the axises and reshape shuffles it around until the wished shape
X, tf.transpose(X), tf.reshape(X, shape=(2,3))

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 3, 5],
        [2, 4, 6]], dtype=int32)>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 2, 3],
        [4, 5, 6]], dtype=int32)>)

In [ ]:
# Try matrix multiplication with transpose rather than reshape
tf.matmul(tf.transpose(X), Y)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 89,  98],
       [116, 128]], dtype=int32)>

In [ ]:
# Create a (3,2) tensor
X = tf.constant([[1,2],
                 [3,4],
                 [5,6]])
# Create another (2,2) tensor
Y = tf.constant([[7,8],
                 [9,10]])
X, Y

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>,
 <tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[ 7,  8],
        [ 9, 10]], dtype=int32)>)

In [ ]:
# this works because the inner dimensions are the same (3,2) (2,2). The inner dimensions are 2 and 2
# And the result has a shape of the outer dimensions (3,2)
X @ Y

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 25,  28],
       [ 57,  64],
       [ 89, 100]], dtype=int32)>

In [ ]:
# Create a (3,2) tensor
X = tf.constant([[1,2],
                 [3,4],
                 [5,6]])
# Create another (2,3) tensor
Y = tf.constant([[7,8,2],
                 [9,10,3]])
X, Y, X @ Y

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[ 7,  8,  2],
        [ 9, 10,  3]], dtype=int32)>,
 <tf.Tensor: shape=(3, 3), dtype=int32, numpy=
 array([[ 25,  28,   8],
        [ 57,  64,  18],
        [ 89, 100,  28]], dtype=int32)>)

**The dot product**

Matrix multiplication is also referred to as the dot product.

You can perform matrix multiplication using:
* `tf.matmul()`
* `tf.tensordot()`

In [ ]:
# Create a (3,2) tensor
X = tf.constant([[1,2],
                 [3,4],
                 [5,6]])
# Create another (3,2) tensor
Y = tf.constant([[7,8],
                 [9,10],
                 [11,12]])
X, Y

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>,
 <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[ 7,  8],
        [ 9, 10],
        [11, 12]], dtype=int32)>)

In [ ]:
# Perform the dot product on X and Y (requires X or Y to be transposed)
tf.tensordot(tf.transpose(X),Y, axes=1)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 89,  98],
       [116, 128]], dtype=int32)>

In [ ]:
# Perform matrix multiplication between X and Y (transposed)
tf.matmul(X, tf.transpose(Y))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 23,  29,  35],
       [ 53,  67,  81],
       [ 83, 105, 127]], dtype=int32)>

In [ ]:
# Perform matrix multiplication between X and Y (reshaped)
tf.matmul(X, tf.reshape(Y, shape=(2,3)))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [ ]:
# Check the values of Y, reshape Y and transposed Y
print("Normal Y: ")
print(Y, "\n") # "\n" is for newline

print("Y reshaped to (2,3): ")
print(tf.reshape(Y, (2,3)), "\n")

print("Y transposed: ")
print(tf.transpose(Y))

Normal Y: 
tf.Tensor(
[[ 7  8]
 [ 9 10]
 [11 12]], shape=(3, 2), dtype=int32) 

Y reshaped to (2,3): 
tf.Tensor(
[[ 7  8  9]
 [10 11 12]], shape=(2, 3), dtype=int32) 

Y transposed: 
tf.Tensor(
[[ 7  9 11]
 [ 8 10 12]], shape=(2, 3), dtype=int32)


Generally, when performing multiplication on two tensors and one of the axes doesn't line up, you will transpose (rather than reshape) one of the tensors to satisfy the matrix multiplication rules.

### Changing the datatype of a tensor



In [ ]:
# Create a new tensor with default datatype (float32)
B = tf.constant([1.7, 7.4])
B, B.dtype

(<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1.7, 7.4], dtype=float32)>,
 tf.float32)

In [ ]:
C = tf.constant([7,10])
C.dtype

tf.int32

In [ ]:
# Change from float32 to float16 (reduced precision)
D = tf.cast(B, dtype=tf.float16)
D, D.dtype

(<tf.Tensor: shape=(2,), dtype=float16, numpy=array([1.7, 7.4], dtype=float16)>,
 tf.float16)

In [ ]:
E = tf.cast(C, dtype=tf.int16)
E, E.dtype

(<tf.Tensor: shape=(2,), dtype=int16, numpy=array([ 7, 10], dtype=int16)>,
 tf.int16)

In [ ]:
# Change form int32 to float32
F = tf.cast(C, dtype=tf.float32)
F, F.dtype

(<tf.Tensor: shape=(2,), dtype=float32, numpy=array([ 7., 10.], dtype=float32)>,
 tf.float32)

### Aggregating tensors

Aggregating tensors = condensing them from multiple values down to a smaller amount of values.

In [ ]:
# Get the absolute values
D = tf.constant([-7, -10])
D, tf.abs(D)

(<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ -7, -10], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 7, 10], dtype=int32)>)

Let's go through the following forms of aggregation:
* Get the minimum
* Get the maximum
* Get the mean of a tensor
* Get the sum of a tensor

In [ ]:
# Create a random tensor with values between 0 and 100 of size 50
E = tf.constant(np.random.randint(0, 100, size=50))
E

<tf.Tensor: shape=(50,), dtype=int64, numpy=
array([27,  9, 43, 77, 18, 40, 74, 19, 59, 76, 72, 15, 55, 43, 56,  5, 14,
       73, 99, 82, 41, 95, 47, 42, 82, 80, 46, 50, 91, 38, 56, 80, 71, 27,
       26,  1, 35, 18,  2, 75, 98, 23, 15, 67, 47, 13, 73, 10, 85, 88])>

In [ ]:
tf.size(E), E.shape, E.ndim

(<tf.Tensor: shape=(), dtype=int32, numpy=50>, TensorShape([50]), 1)

In [ ]:
# Find the minimum
tf.reduce_min(E)

<tf.Tensor: shape=(), dtype=int64, numpy=1>

In [ ]:
# Find the maximum
tf.reduce_max(E)

<tf.Tensor: shape=(), dtype=int64, numpy=99>

In [ ]:
# Find the mean
tf.reduce_mean(E)

<tf.Tensor: shape=(), dtype=int64, numpy=49>

In [ ]:
# Find the sum
tf.reduce_sum(E)

<tf.Tensor: shape=(), dtype=int64, numpy=2478>

🛠️ **Exercise:** With what we've just learned, find the variance and standard deviation of our `E` tensor using TensorFlow methods.

In [ ]:
# Find the variance
print(tf.math.reduce_variance(tf.cast(E, dtype=tf.float64)))

# another way is via tensorflow_probability
import tensorflow_probability as tfp
print(tfp.stats.variance(tf.cast(E, dtype=tf.float64)))

tf.Tensor(824.3664, shape=(), dtype=float64)
tf.Tensor(824.3664, shape=(), dtype=float64)


In [ ]:
# Find the standard deviation
tf.math.reduce_std(tf.cast(E, dtype=tf.float64))

<tf.Tensor: shape=(), dtype=float64, numpy=28.71178155391964>

### Find the positional maximum and minimum



In [ ]:
# Create a new tensor for finding positional minimum and maximum
tf.random.set_seed(42)
F = tf.random.uniform(shape=[50])
F

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
       0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
       0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
       0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
       0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
       0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
       0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
       0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
       0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
       0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
      dtype=float32)>

In [ ]:
# Find the positional maximum
tf.argmax(F)

<tf.Tensor: shape=(), dtype=int64, numpy=42>

In [ ]:
# Index on our largest value position
F[tf.argmax(F)]

<tf.Tensor: shape=(), dtype=float32, numpy=0.967138409614563>

In [ ]:
# Find the max value of F
tf.reduce_max(F)

<tf.Tensor: shape=(), dtype=float32, numpy=0.967138409614563>

In [ ]:
# Check for equality
assert(F[tf.argmax(F)] == tf.reduce_max(F))
F[tf.argmax(F)] == tf.reduce_max(F)

<tf.Tensor: shape=(), dtype=bool, numpy=True>

In [ ]:
# Find the positional minimum
tf.argmin(F)

<tf.Tensor: shape=(), dtype=int64, numpy=16>

In [ ]:
# Find the minimum using the positional minimum index
F[tf.argmin(F)]

<tf.Tensor: shape=(), dtype=float32, numpy=0.009463667869567871>

### Squeezing a tensor (removing all single dimensions)

In [ ]:
# Create a tensor tensor to get started
tf.random.set_seed(42)
G = tf.constant(np.random.randint(0,100,50), shape=(1,1,1,1,50))
G

<tf.Tensor: shape=(1, 1, 1, 1, 50), dtype=int64, numpy=
array([[[[[94, 27, 33,  6, 73, 45, 68, 93, 72, 37, 61, 74, 49, 91,  8,
           74, 62, 57, 33, 35, 16, 60,  7, 12, 46, 35,  3, 19, 95,  6,
           91, 83, 30, 98, 73, 36, 54, 59, 48, 30, 44, 42, 22, 60, 22,
           42, 18,  6, 14, 89]]]]])>

In [ ]:
tf.size(G)

<tf.Tensor: shape=(), dtype=int32, numpy=50>

In [ ]:
G.shape

TensorShape([1, 1, 1, 1, 50])

In [ ]:
G_squeezed = tf.squeeze(G)
G_squeezed, G_squeezed.shape

(<tf.Tensor: shape=(50,), dtype=int64, numpy=
 array([94, 27, 33,  6, 73, 45, 68, 93, 72, 37, 61, 74, 49, 91,  8, 74, 62,
        57, 33, 35, 16, 60,  7, 12, 46, 35,  3, 19, 95,  6, 91, 83, 30, 98,
        73, 36, 54, 59, 48, 30, 44, 42, 22, 60, 22, 42, 18,  6, 14, 89])>,
 TensorShape([50]))

### One-hot encoding tensors



In [ ]:
# Create a list of indices
some_list = [0, 1, 2, 3] # could be red, green, blue, purple

# One hot encode our list of indices
tf.one_hot(some_list, depth=4)

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)>

In [ ]:
# Specify custom values for one hot encoding
tf.one_hot(some_list, depth=4, on_value="yo I love deep learning", off_value="I also like to dance")

<tf.Tensor: shape=(4, 4), dtype=string, numpy=
array([[b'yo I love deep learning', b'I also like to dance',
        b'I also like to dance', b'I also like to dance'],
       [b'I also like to dance', b'yo I love deep learning',
        b'I also like to dance', b'I also like to dance'],
       [b'I also like to dance', b'I also like to dance',
        b'yo I love deep learning', b'I also like to dance'],
       [b'I also like to dance', b'I also like to dance',
        b'I also like to dance', b'yo I love deep learning']],
      dtype=object)>

In [ ]:
# Create a list of indices
some_list = [0, 1, 2, 3] # could be red, green, blue, purple
depth = np.size(some_list)
# One hot encode our list of indices
tf.one_hot(some_list, depth=depth)

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)>

### Squaring, log, square root

In [ ]:
# Create a new tensor
H = tf.range(1, 10)
H

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>

In [ ]:
# Square it
tf.square(H)

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([ 1,  4,  9, 16, 25, 36, 49, 64, 81], dtype=int32)>

In [ ]:
# Find the square root (method require a non-int type)
tf.sqrt(tf.cast(H, dtype=tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([1.       , 1.4142135, 1.7320508, 2.       , 2.2360678, 2.4494896,
       2.6457512, 2.828427 , 3.       ], dtype=float32)>

In [ ]:
# Find the log
tf.math.log(tf.cast(H, dtype=tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([0.       , 0.6931472, 1.0986123, 1.3862944, 1.609438 , 1.7917595,
       1.9459102, 2.0794415, 2.1972246], dtype=float32)>

### Tensor and NumPy

TensorFlow interacts beautifully with NumPy arrays.

🔑 **Note:** One of the main diffenrences between a TensorFlow tensor and a NumPy array is that a TensorFlow tensor can be run on a GPU or TPU (for faster nummerical processing)

In [ ]:
# Create a tensor directly from a NumPy array
J = tf.constant(np.array([3., 7., 10.]))
J

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 3.,  7., 10.])>

In [ ]:
# Convert our tensor back to a NumPy array
np.array(J), type(np.array(J))

(array([ 3.,  7., 10.]), numpy.ndarray)

In [ ]:
# Convert tensor J to a NumPy array
J.numpy(), type(J.numpy())

(array([ 3.,  7., 10.]), numpy.ndarray)

In [ ]:
tf.constant(J)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 3.,  7., 10.])>

In [ ]:
# The default types of each are slightly different
numpy_J = tf.constant(np.array([3., 7., 10.]))
tensor_J = tf.constant([3., 7., 10.])
# Check the datatypes of each
numpy_J.dtype, tensor_J.dtype

(tf.float64, tf.float32)

### Finding access to GPU's


In [ ]:
tf.config.list_physical_devices("CPU")

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [ ]:
tf.config.list_physical_devices("GPU").__len__()

1

In [ ]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
if tf.config.list_physical_devices("GPU"):
  print("GPU Available")
  !nvidia-smi
else:
  print("No GPU")

GPU Available
Sat Mar 15 17:11:23 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   71C    P0             29W /   70W |     106MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+---------------------------------

<function __main__.<lambda>()>

> 🔑 **Note:** If you have access to a CUDA-enabled GPU, TensorFlow will automatically use it whenever possible

### 🛠️ 00. TensorFlow Fundamentals Exercises

1. Create a vector, scalar, matrix and tensor with values of your choosing using tf.constant().
2. Find the shape, rank and size of the tensors you created in 1.
3. Create two tensors containing random values between 0 and 1 with shape [5, 300].
4. Multiply the two tensors you created in 3 using matrix multiplication.
5. Multiply the two tensors you created in 3 using dot product.
6. Create a tensor with random values between 0 and 1 with shape [224, 224, 3].
7. Find the min and max values of the tensor you created in 6 along the first axis.
8. Created a tensor with random values of shape [1, 224, 224, 3] then squeeze it to change the shape to [224, 224, 3].
9. Create a tensor with shape [10] using your own choice of values, then find the index which has the maximum value.
10. One-hot encode the tensor you created in 9.

In [ ]:
# 1. Create a vector, scalar, matrix and tensor with values of your choosing using tf.constant().
vector = tf.constant([1,2,3])
scalar = tf.constant(1)
matrix = tf.constant([[1,2,3],
                      [4,5,6]])
tensor = tf.constant([[[1,2,3],
                       [4,5,6]],
                      [[7,8,9],
                       [10,11,12]]])
vector, scalar, matrix, tensor

(<tf.Tensor: shape=(3,), dtype=int32, numpy=array([1, 2, 3], dtype=int32)>,
 <tf.Tensor: shape=(), dtype=int32, numpy=1>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 2, 3],
        [4, 5, 6]], dtype=int32)>,
 <tf.Tensor: shape=(2, 2, 3), dtype=int32, numpy=
 array([[[ 1,  2,  3],
         [ 4,  5,  6]],
 
        [[ 7,  8,  9],
         [10, 11, 12]]], dtype=int32)>)

In [ ]:
# 2. Find the shape, rank and size of the tensors you created in 1.
print("Shape of vector: ", vector.shape)
print("Shape of scalar: ", scalar.shape)
print("Shape of matrix: ", matrix.shape)
print("Shape of tensor: ", tensor.shape)

Shape of vector:  (3,)
Shape of scalar:  ()
Shape of matrix:  (2, 3)
Shape of tensor:  (2, 2, 3)


In [ ]:
# 3. Create two tensors containing random values between 0 and 1 with shape [5, 300].
tf.random.set_seed(42)
tensor_1 = tf.random.uniform(shape=[5,300])
tensor_2 = tf.random.uniform(shape=[5,300])

tensor_1, tensor_2, tensor_1.shape, tensor_2.shape

(<tf.Tensor: shape=(5, 300), dtype=float32, numpy=
 array([[0.6645621 , 0.44100678, 0.3528825 , ..., 0.31410468, 0.7593535 ,
         0.03699052],
        [0.532024  , 0.29129946, 0.10571766, ..., 0.54052293, 0.31425726,
         0.2200619 ],
        [0.08404207, 0.03614604, 0.97732127, ..., 0.21516645, 0.9786098 ,
         0.00726748],
        [0.7396945 , 0.6653172 , 0.0787828 , ..., 0.7117733 , 0.07013571,
         0.9409125 ],
        [0.15861344, 0.12024033, 0.27218235, ..., 0.8824879 , 0.1432488 ,
         0.44135118]], dtype=float32)>,
 <tf.Tensor: shape=(5, 300), dtype=float32, numpy=
 array([[0.68789124, 0.48447883, 0.9309944 , ..., 0.6920762 , 0.33180213,
         0.9212563 ],
        [0.27369928, 0.10631859, 0.6218617 , ..., 0.4382149 , 0.30427706,
         0.51477313],
        [0.00920248, 0.37280262, 0.8177401 , ..., 0.56786287, 0.49201214,
         0.9892651 ],
        [0.88608265, 0.08672249, 0.12160683, ..., 0.91770685, 0.72545695,
         0.8280058 ],
        [0.36690

In [ ]:
# 4. Multiply the two tensors you created in 3 using matrix multiplication.
tensor_1 @ tf.transpose(tensor_2)

<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[80.333435, 73.40498 , 77.15961 , 73.98368 , 80.90053 ],
       [75.14637 , 68.80438 , 74.24303 , 71.84184 , 75.60205 ],
       [79.7594  , 75.64456 , 77.79758 , 74.748726, 80.55984 ],
       [75.085266, 69.06408 , 74.30776 , 72.27615 , 76.05667 ],
       [85.05689 , 74.26629 , 78.00687 , 74.886795, 83.13417 ]],
      dtype=float32)>

In [ ]:
tf.matmul(tensor_1, tf.transpose(tensor_2))

<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[80.333435, 73.40498 , 77.15961 , 73.98368 , 80.90053 ],
       [75.14637 , 68.80438 , 74.24303 , 71.84184 , 75.60205 ],
       [79.7594  , 75.64456 , 77.79758 , 74.748726, 80.55984 ],
       [75.085266, 69.06408 , 74.30776 , 72.27615 , 76.05667 ],
       [85.05689 , 74.26629 , 78.00687 , 74.886795, 83.13417 ]],
      dtype=float32)>

In [ ]:
# 5. Multiply the two tensors you created in 3 using dot product.
tf.tensordot(tensor_1, tf.transpose(tensor_2), axes=1)

<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[80.333435, 73.40498 , 77.15961 , 73.98368 , 80.90053 ],
       [75.14637 , 68.80438 , 74.24303 , 71.84184 , 75.60205 ],
       [79.7594  , 75.64456 , 77.79758 , 74.748726, 80.55984 ],
       [75.085266, 69.06408 , 74.30776 , 72.27615 , 76.05667 ],
       [85.05689 , 74.26629 , 78.00687 , 74.886795, 83.13417 ]],
      dtype=float32)>

In [ ]:
#6. Create a tensor with random values between 0 and 1 with shape [224, 224, 3].
tf.random.set_seed(42)
tensorX = tf.random.uniform(shape=[224,224,3])
tensorX

<tf.Tensor: shape=(224, 224, 3), dtype=float32, numpy=
array([[[0.6645621 , 0.44100678, 0.3528825 ],
        [0.46448255, 0.03366041, 0.68467236],
        [0.74011743, 0.8724445 , 0.22632635],
        ...,
        [0.42612267, 0.09686017, 0.16105258],
        [0.1487099 , 0.04513884, 0.9497483 ],
        [0.4393103 , 0.28527975, 0.96971095]],

       [[0.73308516, 0.5657046 , 0.33238935],
        [0.8838178 , 0.87544763, 0.56711245],
        [0.8879347 , 0.47661996, 0.42041814],
        ...,
        [0.7716515 , 0.9116473 , 0.3229897 ],
        [0.43050945, 0.83253574, 0.45549798],
        [0.29816985, 0.9639522 , 0.3316357 ]],

       [[0.41132426, 0.2179662 , 0.53570235],
        [0.5112119 , 0.6484759 , 0.8894886 ],
        [0.42459428, 0.20189774, 0.85781324],
        ...,
        [0.02888799, 0.3995477 , 0.11355484],
        [0.68524575, 0.04945195, 0.17778492],
        [0.97627187, 0.79811585, 0.9411576 ]],

       ...,

       [[0.9019445 , 0.27011132, 0.8090267 ],
        [0.32

In [ ]:
# 7. Find the min and max values of the tensor you created in 6 along the first axis.
print("Min: ", tf.reduce_min(tensorX))
print("Max: ", tf.reduce_max(tensorX))

Min:  tf.Tensor(3.5762787e-07, shape=(), dtype=float32)
Max:  tf.Tensor(0.999998, shape=(), dtype=float32)


In [ ]:
# 8. Created a tensor with random values of shape [1, 224, 224, 3] then squeeze it to change the shape to [224, 224, 3].
tf.random.set_seed(42)
tensorY = tf.random.uniform(shape=[1,224,224,3])
tensorY.shape

TensorShape([1, 224, 224, 3])

In [ ]:
tf.squeeze(tensorY).shape

TensorShape([224, 224, 3])

In [ ]:
# 9. Create a tensor with shape [10] using your own choice of values, then find the index which has the maximum value.
tensorCustom = tf.constant([1,2,3,4,5,6,7,8,9,10])
tensorCustom

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=int32)>

In [ ]:
tf.argmax(tensorCustom)

<tf.Tensor: shape=(), dtype=int64, numpy=9>

In [ ]:
tensorCustom[tf.argmax(tensorCustom)]

<tf.Tensor: shape=(), dtype=int32, numpy=10>

In [ ]:
# 10. One-hot encode the tensor you created in 9.
tf.one_hot(tensorCustom, depth=np.size(tensorCustom))

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>